# Model Monitoring & Model Card

In this exercise we will simulate an environment where we have model training data from multi day runs for our previoulsy built model, and explore Vetiver's monitoring functions.
<br>
We will also use the model card framework from vetiver to build a model card for this workflow.

```bash
# deactivate older venv
deactivate 

# Create a virtual environment
# Use our alias!
py-venv
python -m pip install -r requirements.txt

```

In [ ]:
# import required libraries
import os
from datetime import datetime, timedelta

import ibis
import numpy as np
import pandas as pd
import pins
import requests
import rsconnect
import vetiver
import xgboost as xgb
from dotenv import load_dotenv
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sqlalchemy import create_engine
import re

### Task 1 - Build mulitple days training data

#### 🔄 Task 

This is a simulation exercise as we do not have training data from future runs of the model. <br> Use the code provided below to generate a simulated dataset for this workflow

#### ✅ Solution

In [2]:
# # Simulate multiple days of input data
data = X_test.copy()
data["RESULTS"] = y_test
data["date_obs"] = ""
data

In [ ]:
# # to make predictions, change the endpoint to your model
endpoint = ""
data["preds"] = vetiver.predict(endpoint, data).iloc[:, 0].values


In [ ]:
# # add last three dates
day_list = [
    (pd.Timestamp.today() - timedelta(days=2)).strftime("%Y-%m-%d"),
    (pd.Timestamp.today() - timedelta(days=1)).strftime("%Y-%m-%d"),
    (pd.Timestamp.today().strftime("%Y-%m-%d")),
]
day_list

data["date_obs"] = np.random.choice(day_list, size=len(data))


In [ ]:
# pin this dataset for use in model card
board.pin_write(data,"gagan/inspection_results_monitoring",type="parquet")


### Task 2 - Build model monitoring metrics and explore further deployment

#### 🔄 Task 

We will now use the metrics compute feature in Vetiver to build a moniotring stratedy.
- Identify which metrics to monitor
- Calculate the said metrics
- Save the results as pin

Once we have a verison of the metrics deployed, explore how we can further this step in a real world situation using Posit Connect's scheduling feature

#### ✅ Solution

In [ ]:
# define which metrics to track
metric_set = [metrics.mean_absolute_error, metrics.mean_squared_error, metrics.r2_score]

# choose time period to track over
td = timedelta(days=1)

# calculate metrics
original_metrics = vetiver.compute_metrics(
    data=data,
    date_var="date_obs",
    period=td,
    metric_set=metric_set,
    truth="RESULTS",
    estimate="preds",
)

# Write metrics as a pin
model_board.pin_write(original_metrics, "gagan/inspection_model_metrics", type="csv")

### Task 3 - Building a model card

#### 🔄 Task 

Use *vetiver.model_card* function to generate a model card sample.
<br>Open the sample and explore what sections are there. Then, edit the card with relavant information of our example and deploy on Posit Connect

In [ ]:
vetiver.model_card()